In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
from matplotlib import colors as mcolors
import random
import pickle

from sklearn.metrics import fbeta_score
from argparse import Namespace

from scipy.stats import entropy
%matplotlib inline
sys.path.append('../src/')

from novelty_dfm_CL.novelty_utils import confidence_scores, Threshold_n_Select_Simple
from novelty_dfm_CL.novelty_plot_utils import confidence_multi_task_subplots, multi_task_subplots, evaluate_subset_n_complement, ThresholdingResults



# Set parameters for evaluation

In [6]:
# 0) Data results experiment of CL_DFM.ipynb
dir_results = '../src/novelty_dfm_CL/Results_DFM_CL/cifar10/101121_123634/'

# f-beta score parameter
f1_beta = 1

chosen_threshold = 60 # current file uses this percentile threhold for simple threhsolding of novel/old

# Load Scores + Ground Truth per task

In [7]:
with open('%s/results_tasks.pickle'%(dir_results), 'rb') as handle:
    per_task_results = pickle.load(handle)

if isinstance(per_task_results, dict):
    per_task_results = Namespace(**per_task_results)

num_tasks = len(per_task_results.gt.keys())

# Compute Stats

In [4]:
percentiles = np.arange(20,100,5)

Results = ThresholdingResults(percentiles, num_tasks)

Results.log_results(per_task_results)



# Plot Precision, Recall and F-beta x Thresholding by percentiles

In [5]:

data_perform = np.stack((Results.precision_novelty, Results.recall_novelty, Results.fbeta_novelty, Results.accuracy_novelty), axis=0)

stats_names = ['precision', 'recall', 'F_%.2f'%f1_beta, 'accuracy']

colors_set = ['r', 'b', 'g', 'c']


savepath = dir_results + '/performance_vs_thresholding_percentiles.jpg'

multi_task_subplots(data_perform, savepath=savepath, title_plot='Performance when Thresholding', y_ticks=np.arange(0,1.15, 0.1), xlabels=np.tile(percentiles,(num_tasks,1)).T, 
                    fig_size_=(15,25),  codes=stats_names, colors_set=colors_set, stepsize=1, task_delim=False, turnoffX=False)

# Plot confidence values (Novel x Old) x Thresholds 

In [6]:
# TODO change this to bar plot (better visualization)


savepath = dir_results + '/confidences_vs_thresholding_percentiles_variance.jpg'


th_mean, th_e = confidence_multi_task_subplots([Results.confidences_Novel, Results.confidences_Old], percentiles, savepath=savepath, title_plot='Confidences when Thresholding',\
     fig_size_=(10.5,20),  codes=['TP-Novel', 'FP-Old'], stepsize_task=1, colors_set=['c', 'm'])

# confidence_multi_task_subplots([confidences_Novel, confidences_Old], title_plot='COnfidences x Threshold', y_step=0.01, y_ticks=np.arange(0.5,3.0,0.5), fig_size_=(10.5,20),  codes=['TP-Novel', 'FP-Old'], stepsize_task=1, colors_set=['c', 'm'])

/home/amandari/miniconda3/envs/remind_proj/lib/python3.7/site-packages/numpy/core/_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/amandari/miniconda3/envs/remind_proj/lib/python3.7/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ValueError: arange: cannot compute length

# Evaluate Threshold per task

In [ ]:

# task=7

# indices  = [Threshold_n_Select(task, per_task_results, th_percentile_score=th, th_percentile_confidence=-1,  metric_confidence=('variance', 'min'))[0] for th in percentiles]

# precisions_n = [evaluate_subset_n_complement(task, per_task_results, inds)[0][0] for inds in indices]
# precisions_o = [evaluate_subset_n_complement(task, per_task_results, inds)[1][0] for inds in indices]

# recalls_n = [evaluate_subset_n_complement(task, per_task_results, inds)[0][1] for inds in indices]
# recalls_o = [evaluate_subset_n_complement(task, per_task_results, inds)[1][1] for inds in indices]

# accuracies_n = [evaluate_subset_n_complement(task, per_task_results, inds)[0][-2] for inds in indices]
# accuracies_o = [evaluate_subset_n_complement(task, per_task_results, inds)[1][-2] for inds in indices]


# %matplotlib inline
# plt.plot(np.arange(20,100,5), precisions_n)
# plt.plot(np.arange(20,100,5), recalls_n, 'r')
# plt.ylim([0,1.0])